In [1]:
import h5py
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
#import math
#import random
import heapq
import matplotlib.pyplot as plt
import random

In [73]:
filename = '/usr/local/home/yizhu/Desktop/BSLB_sample.h5'

f = h5py.File(filename,'r+') 
print(f['traces'].shape)
f.close()

(190500, 1623)


In [74]:

filename = '/usr/local/home/yizhu/Desktop/Yizhu/BFFile_withPython_save/cswap_pointer.h5'

f = h5py.File(filename,'r+') 
data_ecc_pointer = f['attacking_data'][:,0]
traces_ecc_pointer = f['attacking_traces']
print(traces_ecc_pointer.shape)
f.close()


(12750, 1000)


In [3]:
class Cluster_k_means(object):

    def __init__(self, traces, k, step):

        self.found = 0  #iretation count
        
        self.step = step  #iretation limit
        self.k = k
        self.traces = traces
        self.dim = self.traces.shape[1]

        index_random = set() 
        while(len(index_random)<k):
            index_random.add(random.randint(0,self.traces.shape[0]-1))
        index_random = list(index_random)

        self.guesses = [list(self.traces[index_random[i]]) for i in range(k)] #initial center

        self.initial_convert_and_sort()

    def initial_convert_and_sort(self):

        self.traces = self.get_data(self.traces)
        self.unpack_new_centers()

    def get_data(self, traces): #classification by picking minimum distance
        new_data = []

        for trace in traces:
            distances = []
            for guess in self.guesses:
                distance = math.sqrt(
                    sum((trace[i] - guess[i])**2 for i in range(self.dim))
                )
                distances.append(distance)

            ki = distances.index(min(distances))

            d = {
                "ki": ki,
                "trace": trace
            }
            new_data.append(d)

        return new_data
    
    def unpack_new_centers(self):  # update center
        for ki, guess in enumerate(self.guesses):

            traces = [d["trace"] for d in self.traces if d["ki"] == ki]
            new_guess = []

            if traces:
                for i in range(self.dim):
                    iguess = sum(trace[i] for trace in traces) / len(traces)
                    new_guess.insert(i, iguess)

                self.guesses[ki] = new_guess
            else:
                mini = lambda i: min(d["trace"][i] for d in self.data)
                maxi = lambda i: max(d["trace"][i] for d in self.data)
                self.guesses[ki] = [random.triangular(mini(i), maxi(i)) for i in range(self.dim)]

        self.correspond_nearest()
    
    def correspond_nearest(self): # 

        traces = [d["trace"] for d in self.traces]
        new_data = self.get_data(traces)

        if new_data == self.traces:
            self.found += 1
            if self.found >= self.step:
                print (self.found)
                return self.guesses
            else:
                self.unpack_new_centers()
        else:
            self.traces = new_data
            self.unpack_new_centers()

#mean_data = Cluster_k_means(f['traces'][:10],2,10).guesses


In [61]:
# compute sum-of-squared-t values

class SOST():
    def __init__(self, traces, labels):
        self.traces = traces
        self.trace_length = self.traces.shape[1]
        self.trace_number= self.traces.shape[0]
        self.lables = labels
        self.classfication = 2
        
        indices = self.list_of_indices_per_classfication()
        self.sost = self.sost_coefficients(indices)

    def list_of_indices_per_classfication(self):

        indices_list = []
        for i in range(self.classfication):
            indices_list.append([])
            for j in range(self.trace_number):
                if self.lables[j] == i:
                    indices_list[i].append(j)

        return indices_list
    
    def sost_coefficients(self, indices):
        sost_coefficients = np.empty(self.trace_length)
        for t in range(self.trace_length):
            means = np.empty(self.classfication)
            variances = np.empty(self.classfication)
            counts = np.empty(self.classfication)
            result = 0

            for i in range(self.classfication):
                values = self.traces[indices[i], t]
                means[i] = np.mean(values)
                variances[i] = np.var(values)
                counts[i] = len(values)

            if self.classfication == 2:
                temp = pow(means[0] - means[1], 2)
                temp /= variances[0]/counts[0] + variances[1]/counts[1]
                result += temp
            else: 
                print('only compute 2 classifications!')
                return

            sost_coefficients[t] = result

        return sost_coefficients
     

In [5]:
# outlier detection

def outlier_remove(data, traces, threshold):

    means = traces.mean(0)
    stds = traces.std(0)
    traces_evl_value = (abs(traces-means)).sum(1) - (threshold * stds).sum()
    traces_evl = traces_evl_value >= 0

    outliers_index = [i for i in range(traces.shape[0]) if traces_evl[i]]
    print('outlier number:', len(outliers_index))
    print('outlier ratio:',len(outliers_index)/traces.shape[0])

    traces_new = np.delete(traces, outliers_index, 0)
    print('traces_new.shape:', traces_new.shape)

    data_new = np.delete(data, outliers_index, 0)
    print('data_new.shape:', data_new.shape)

    return data_new, traces_new


In [62]:

def outlier_replace(traces, threshold):

    means = traces.mean(0)
    stds = traces.std(0)
    traces_evl_value = abs(traces-means) - threshold * stds
    traces_evl = traces_evl_value > 0
    
    #means_evl = 1- traces_evl 

    #means_no_outliers = (traces * means_evl).mean(0)
    for t in range(traces.shape[0]):
        for i in range(traces.shape[1]):
            if traces_evl[t][i]:
                #traces[t][i] = means_no_outliers[i]
                traces[t][i] = means[i]

    print('outlier number:', traces_evl.sum())
    print('outlier ratio:',traces_evl.sum()/(traces.shape[0]*traces.shape[1]))
    print('traces_new.shape:', traces.shape)
    return traces

In [67]:
def attampt(labels, traces, POI_numbers, steps,note):
    # save results
    data_save = np.empty((3,steps))

    if len(POI_numbers) + 1 < steps:
        raise ValueError

    kmeans_cluster = KMeans(n_clusters=2, random_state=0).fit(traces)

    kmeans_labels = kmeans_cluster.labels_ 
    #recovery_rate = (kmeans_labels == labels).sum()/len(labels)
    recovery_rate = (np.multiply((kmeans_labels == labels),1)).sum()/len(labels)
    if recovery_rate < 0.5:
        recovery_rate = 1 - recovery_rate

    silhouette_score = metrics.silhouette_score(traces, kmeans_labels, metric='euclidean')
    #print(f'step 1: recovery {recovery_rate}, Silhouette {silhouette_score}'  )

    # save results
    data_save[0,0] = str(traces.shape[1])
    data_save[1,0] = str(recovery_rate)
    data_save[2,0] = str(silhouette_score)
 
    for i in range(steps-1):
        sost_all = SOST(traces,kmeans_cluster.labels_).sost
        POI_number = POI_numbers[i]

        POI_indexs = np.argsort(-sost_all)[:POI_number]
        traces_new = traces[ :,POI_indexs]
        kmeans_cluster = KMeans(n_clusters=2, random_state=0).fit(traces_new)

        kmeans_labels = kmeans_cluster.labels_ 
        #recovery_rate = (kmeans_labels == labels).sum()/len(labels)
        recovery_rate = (np.multiply((kmeans_labels == labels),1)).sum()/len(labels)
        if recovery_rate < 0.5:
            recovery_rate = 1 - recovery_rate

        #silhouette_score = metrics.silhouette_score(traces, kmeans_labels, metric='euclidean')
        silhouette_score = metrics.silhouette_score(traces_new, kmeans_labels, metric='euclidean')
        #print(f'step {i+2}: recovery {recovery_rate}, Silhouette {silhouette_score}')

        data_save[0,i+1] = str(POI_number)
        data_save[1,i+1] = str(recovery_rate)
        data_save[2,i+1] = str(silhouette_score)

        if  POI_number == 1:
            data_save[2,i+1] = POI_indexs


    fmt = '%d', '%1.3f', '%1.3f'
    np.savetxt(f"ecc_pointer/multi-{traces.shape}-{note}-{steps}.txt", data_save.T, fmt=fmt, delimiter=' ')


In [9]:
def kmeans_single_dim(traces): 
    n_traces = traces.shape[0]
    n_points = traces.shape[1]

    labels = np.zeros((n_traces, n_points))
    final_labels = np.zeros(n_traces)

    # apply kmeans to all points of one timing, determine the label of each point
    for i in range(traces.shape[1]):
        kmeans_cluster = KMeans(n_clusters=2, random_state=0).fit(traces[:, i].reshape(-1, 1))
        labels[:,i] = kmeans_cluster.labels_

    # reference timing (one column), random choose(less meaning) or choose leakage 
    t = int(0.7 * n_points)
    t = 0
    reference_t = labels[:, t]

    # flip it if one column is far different from the reference column
    for i in range(n_points):
        eval = labels[:, i] == reference_t
        if eval.sum() < n_traces/2:
            labels[:, i] = 1 - labels[:, i]

    # majority rule to determine the label of each trace
    for i in range(n_traces):
        if list(labels[i]).count(1) > n_points / 2:
            final_labels[i] = 1

    return final_labels



In [10]:
def attampt_single_dim(labels, traces, POI_numbers, steps, note):
    # save results
    data_save = np.empty((3,steps))

    if len(POI_numbers) + 1 < steps:
        raise ValueError

    
    kmeans_labels = kmeans_single_dim(traces)
 
    recovery_rate = (kmeans_labels == labels).sum()/len(labels)
    if recovery_rate < 0.5:
        recovery_rate = 1 - recovery_rate

    silhouette_score = metrics.silhouette_score(traces, kmeans_labels, metric='euclidean')
    #print(f'step 1: recovery {recovery_rate}, Silhouette {silhouette_score}'  )

    # save results
    data_save[0,0] = str(traces.shape[1])
    data_save[1,0] = str(recovery_rate)
    data_save[2,0] = str(silhouette_score)


    for i in range(steps-1):
        sost_all = SOST(traces,kmeans_labels).sost
        POI_number = POI_numbers[i]

        POI_indexs = np.argsort(-sost_all)[:POI_number]
        traces_new = traces[ :,POI_indexs]
        kmeans_labels = kmeans_single_dim(traces_new)

        recovery_rate = (kmeans_labels == labels).sum()/len(labels)
        if recovery_rate < 0.5:
            recovery_rate = 1 - recovery_rate

        #silhouette_score = metrics.silhouette_score(traces, kmeans_labels, metric='euclidean')
        ilhouette_score = metrics.silhouette_score(traces_new, kmeans_labels, metric='euclidean')
        #print(f'step {i+2}: recovery {recovery_rate}, Silhouette {silhouette_score}'  )

        data_save[0,i+1] = str(POI_number)
        data_save[1,i+1] = str(recovery_rate)
        data_save[2,i+1] = str(silhouette_score)

    fmt = '%d', '%1.3f', '%1.3f'
    np.savetxt(f"single/single-{traces.shape}-{steps}-{note}.txt", data_save.T, fmt=fmt, delimiter=' ')
    

In [8]:
def attampt_single(labels, traces, POI_numbers, steps, note, labels_all):

    data_save = np.empty((3,steps))

    if len(POI_numbers) + 1 < steps:
        raise ValueError
    
    n_traces = traces.shape[0]
    n_points = traces.shape[1]

    # reference timing (one column), random choose(less meaning) or choose leakage 
    random.seed(2023)
    t = random.randint(0, n_points-1)
    reference_t = labels_all[:, t]

    kmeans_labels = labels_base_reference(reference_t, labels_all, n_points, n_traces)



    recovery_rate = (kmeans_labels == labels).sum()/len(labels)
    if recovery_rate < 0.5:
        recovery_rate = 1 - recovery_rate

    silhouette_score = metrics.silhouette_score(traces, kmeans_labels, metric='euclidean')
    #print(f'step 1: recovery {recovery_rate}, Silhouette {silhouette_score}'  )

    # save results
    data_save[0,0] = str(traces.shape[1])
    data_save[1,0] = str(recovery_rate)
    data_save[2,0] = str(silhouette_score)


    for i in range(steps-1):
        sost_all = SOST(traces,kmeans_labels).sost
        POI_number = POI_numbers[i]

        POI_indexs = np.argsort(-sost_all)[:POI_number]
        traces_new = traces[ :,POI_indexs]
        n_traces = traces_new.shape[0]
        n_points = traces_new.shape[1]

        t = random.randint(0, n_points-1)
        reference_t = traces_new[:, t]

        kmeans_labels = labels_base_reference(reference_t, labels_all[ :,POI_indexs], n_points, n_traces)

        recovery_rate = (kmeans_labels == labels).sum()/len(labels)
        if recovery_rate < 0.5:
            recovery_rate = 1 - recovery_rate

        #silhouette_score = metrics.silhouette_score(traces, kmeans_labels, metric='euclidean')
        silhouette_score = metrics.silhouette_score(traces_new, kmeans_labels, metric='euclidean')
        #print(f'step {i+2}: recovery {recovery_rate}, Silhouette {silhouette_score}'  )

        data_save[0,i+1] = str(POI_number)
        data_save[1,i+1] = str(recovery_rate)
        data_save[2,i+1] = str(silhouette_score)

    fmt = '%d', '%1.3f', '%1.3f'
    np.savetxt(f"single/random-{traces.shape}-{steps}-{note}.txt", data_save.T, fmt=fmt, delimiter=' ')


In [12]:
def labels_base_reference(reference_t, labels, n_points, n_traces):
    final_labels = np.zeros(n_traces)

    # flip it if one column is far different from the reference column
    for i in range(n_points):
        eval = labels[:, i] == reference_t
        if eval.sum() < n_traces/2:
            labels[:, i] = 1 - labels[:, i]

    # majority rule to determine the label of each trace
    for i in range(n_traces):
        if list(labels[i]).count(1) > n_points / 2:
            final_labels[i] = 1

    return final_labels

In [ ]:

attampt(f['data'][:10000], f['traces'][:10000], [300], 2, 300)

In [ ]:
POIs = [600 for i in range(8)]
print(POIs)

In [ ]:
attampt(f['data'][:10000], f['traces'][:10000], POIs, len(POIs)+1, '600')

In [ ]:
initial = 1623
POIs = [initial]
rate = 0.5
for i in range(80):
    POI = POIs[-1]
    temp = int(POI * rate)
    if temp > 0:
        POIs.append(temp)
        rate_temp = rate * 1.05
        if rate_temp < 1:
            rate = rate_temp
    else:
        continue        

    if POIs:
        attampt(f['data'][:10000], f['traces'][:10000], POIs, len(POIs)+1, f'{initial}-increasing rate')
    


In [ ]:
initial = 1623
POIs = [initial]
rate = 0.9
for i in range(80):
    POI = POIs[-1]
    temp = int(POI * rate)
    if temp > 0:
        POIs.append(temp)
        rate_temp = rate * 1.1
        if rate_temp < 1:
            rate = rate_temp
    else:
        continue        

    if POIs:
        attampt(f['data'][:10000], f['traces'][:10000], POIs, len(POIs)+1, f'{initial}-increasing rate')

In [ ]:
initials = [1623]
rates = [0.9]
for initial in initials:
    
    for rate in rates:
        POIs = [initial]
        for i in range(80):
            POI = int(initial * rate ** (i+1) )

            if POI > 0:
                POIs.append(POI)
            else:
                continue

        if POIs:
            attampt(f['data'][:10000], f['traces'][:10000], POIs, len(POIs)+1, f'{initial}-{rate}-reference')

In [69]:
def get_result(data, traces, steps, memo):
    attampt(data, traces, [500 for i in range(steps-1)], steps, f'500-{memo}')
    attampt(data, traces, [400 for i in range(steps-1)], steps, f'400-{memo}')
    attampt(data, traces, [300 for i in range(steps-1)], steps, f'300-{memo}')
    attampt(data, traces, [200 for i in range(steps-1)], steps, f'200-{memo}')
    attampt(data, traces, [150 for i in range(steps-1)], steps, f'150-{memo}')
    attampt(data, traces, [100 for i in range(steps-1)], steps, f'100-{memo}')
    attampt(data, traces, [70 for i in range(steps-1)], steps, f'70-{memo}')
    attampt(data, traces, [50 for i in range(steps-1)], steps, f'50-{memo}')
    attampt(data, traces, [30 for i in range(steps-1)], steps, f'30-{memo}')
    attampt(data, traces, [20 for i in range(steps-1)], steps, f'20-{memo}')
    attampt(data, traces, [10 for i in range(steps-1)], steps, f'10-{memo}')
    attampt(data, traces, [5 for i in range(steps-1)], steps, f'5-{memo}')



In [105]:
data_no_outliers , traces_no_outliers = outlier_remove(f['data'][:10000], f['traces'][:10000],1)
print('\n')

outlier number: 1234
outlier ratio: 0.1234
traces_new.shape: (8766, 1623)
data_new.shape: (8766,)




In [ ]:
def get_result_no(data, traces, steps):
    attampt(data, traces, [500 for i in range(steps-1)], steps, '500-no_outliers-1')
    attampt(data, traces, [400 for i in range(steps-1)], steps, '400-no_outliers-1')
    attampt(data, traces, [300 for i in range(steps-1)], steps, '300-no_outliers-1')
    attampt(data, traces, [200 for i in range(steps-1)], steps, '200-no_outliers-1')
    attampt(data, traces, [150 for i in range(steps-1)], steps, '150-no_outliers-1')
    attampt(data, traces, [100 for i in range(steps-1)], steps, '100-no_outliers-1')
    attampt(data, traces, [70 for i in range(steps-1)], steps, '70-no_outliers-1')
    attampt(data, traces, [50 for i in range(steps-1)], steps, '50-no_outliers-1')
    attampt(data, traces, [30 for i in range(steps-1)], steps, '30-no_outliers-1')
    attampt(data, traces, [20 for i in range(steps-1)], steps, '20-no_outliers-1')
    attampt(data, traces, [10 for i in range(steps-1)], steps, '10-no_outliers-1')
    attampt(data, traces, [5 for i in range(steps-1)], steps, '5-no_outliers-1')

In [ ]:
get_result_no(data_no_outliers, traces_no_outliers, 20)

In [ ]:
#data_no_outliers , traces_no_outliers = outlier_remove(f['data'][:10000], f['traces'][:10000],1)
#print('\n')
#get_result_no(data_no_outliers, traces_no_outliers, 20)

initials = [1623]
rates = [0.9 - 0.1*i  for i in range(9)]
for initial in initials:
    for rate in rates:
        POIs = [initial]
        for i in range(88):
            POI = int(initial * rate ** (i+1) )

            if POI > 0:
                POIs.append(POI)
            else:
                continue

        if POIs:
            attampt(data_no_outliers, traces_no_outliers, POIs, len(POIs)+1, f'{initial}-{rate}-no_outliers-1-reference')

In [39]:
def get_result_replace(data, traces, steps, threshold):
    #attampt(data, traces, [500 for i in range(steps-1)], steps, f'500-replace_outliers-{threshold}')
    #attampt(data, traces, [400 for i in range(steps-1)], steps, f'400-replace_outliers-{threshold}')
    #attampt(data, traces, [300 for i in range(steps-1)], steps, f'300-replace_outliers-{threshold}')
    #attampt(data, traces, [200 for i in range(steps-1)], steps, f'200-replace_outliers-{threshold}')
    #attampt(data, traces, [150 for i in range(steps-1)], steps, f'150-replace_outliers-{threshold}')
    attampt(data, traces, [100 for i in range(steps-1)], steps, f'100-replace_outliers-{threshold}')
    #attampt(data, traces, [70 for i in range(steps-1)], steps, f'70-replace_outliers-{threshold}')
    #attampt(data, traces, [50 for i in range(steps-1)], steps, f'50-replace_outliers-{threshold}')
    attampt(data, traces, [30 for i in range(steps-1)], steps, f'30-replace_outliers-{threshold}')
    #attampt(data, traces, [20 for i in range(steps-1)], steps, f'20-replace_outliers-{threshold}')
    #attampt(data, traces, [10 for i in range(steps-1)], steps, f'10-replace_outliers-{threshold}')
    attampt(data, traces, [5 for i in range(steps-1)], steps, f'5-replace_outliers-{threshold}')




In [71]:
traces_mean_outliers = outlier_replace(traces_ecc_pointer[:10000], 2)
print('\n')


outlier number: 330799
outlier ratio: 0.0330799
traces_new.shape: (10000, 1000)




In [68]:

#get_result_replace(f['data'][:10000], traces_mean_outliers, 20)

initials = [1623]
rates = [0.9 - 0.1*i  for i in range(9)]
#rates = [0.9]
for initial in initials:
    for rate in rates:
        POIs = [initial]
        for i in range(88):
            POI = int(initial * rate ** (i+1) )

            if POI > 0:
                POIs.append(POI)
            else:
                continue

        if POIs:
            attampt(data_ecc_pointer[:10000], traces_ecc_pointer[:10000], POIs, len(POIs)+1, f'{initial}-{rate}')

In [72]:
get_result(data_ecc_pointer[:10000], traces_mean_outliers, 20, 'replace_outliers-2')

In [15]:
def get_result_removed(data, traces, steps, threshold):
    n_traces = traces.shape[0]
    n_points = traces.shape[1]

    labels_all = np.zeros((n_traces, n_points))
    

    # apply kmeans to all points of one timing, determine the label of each point
    for i in range(traces.shape[1]):
        kmeans_cluster = KMeans(n_clusters=2, random_state=0).fit(traces[:, i].reshape(-1, 1))
        labels_all[:,i] = kmeans_cluster.labels_

    #attampt_single(data, traces, [500 for i in range(steps-1)], steps, f'500-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [400 for i in range(steps-1)], steps, f'400-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [300 for i in range(steps-1)], steps, f'300-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [200 for i in range(steps-1)], steps, f'200-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [150 for i in range(steps-1)], steps, f'150-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [100 for i in range(steps-1)], steps, f'100-no_outliers-{threshold}', labels_all)
    #ttampt_single(data, traces, [70 for i in range(steps-1)], steps, f'70-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [50 for i in range(steps-1)], steps, f'50-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [30 for i in range(steps-1)], steps, f'30-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [20 for i in range(steps-1)], steps, f'20-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [10 for i in range(steps-1)], steps, f'10-no_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [5 for i in range(steps-1)], steps, f'5-no_outliers-{threshold}', labels_all)

    attampt_single(data, traces, [500 for i in range(steps-1)], steps, '500', labels_all)
    #attampt_single(data, traces, [400 for i in range(steps-1)], steps, '400', labels_all)
    #attampt_single(data, traces, [300 for i in range(steps-1)], steps, '300', labels_all)
    #ttampt_single(data, traces, [200 for i in range(steps-1)], steps, '200', labels_all)
    attampt_single(data, traces, [150 for i in range(steps-1)], steps, '100', labels_all)
    #attampt_single(data, traces, [100 for i in range(steps-1)], steps, '70', labels_all)
    #attampt_single(data, traces, [70 for i in range(steps-1)], steps, '50', labels_all)
    #attampt_single(data, traces, [50 for i in range(steps-1)], steps, '40', labels_all)
    attampt_single(data, traces, [30 for i in range(steps-1)], steps, '30', labels_all)
    #attampt_single(data, traces, [20 for i in range(steps-1)], steps, '20', labels_all)
    #attampt_single(data, traces, [10 for i in range(steps-1)], steps, '10', labels_all)
    #attampt_single(data, traces, [5 for i in range(steps-1)], steps, '', labels_all)
    

In [16]:


#data_no_outliers , traces_no_outliers = outlier_remove(f['data'][:10000], f['traces'][:10000],1.1)
print('\n')
get_result_removed(f['data'][:10000], f['traces'][:10000], 20, 0)


/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/

In [16]:
traces_mean_outliers = outlier_replace(f['traces'][:10000], 2)
print('\n')
get_result_replace(f['data'][:10000], traces_mean_outliers, 20, 2)


outlier number: 749175
outlier ratio: 0.04615988909426987
traces_new.shape: (10000, 1623)




/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/

In [15]:
def get_result_replace(data, traces, steps, threshold):
    n_traces = traces.shape[0]
    n_points = traces.shape[1]

    labels_all = np.zeros((n_traces, n_points))
    

    # apply kmeans to all points of one timing, determine the label of each point
    for i in range(traces.shape[1]):
        kmeans_cluster = KMeans(n_clusters=2, random_state=0).fit(traces[:, i].reshape(-1, 1))
        labels_all[:,i] = kmeans_cluster.labels_

    attampt_single(data, traces, [500 for i in range(steps-1)], steps, f'500-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [400 for i in range(steps-1)], steps, f'400-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [300 for i in range(steps-1)], steps, f'300-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [200 for i in range(steps-1)], steps, f'200-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [150 for i in range(steps-1)], steps, f'150-replace_outliers-{threshold}', labels_all)
    attampt_single(data, traces, [100 for i in range(steps-1)], steps, f'100-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [70 for i in range(steps-1)], steps, f'70-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [50 for i in range(steps-1)], steps, f'50-replace_outliers-{threshold}', labels_all)
    attampt_single(data, traces, [30 for i in range(steps-1)], steps, f'30-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [20 for i in range(steps-1)], steps, f'20-replace_outliers-{threshold}', labels_all)
    #attampt_single(data, traces, [10 for i in range(steps-1)], steps, f'10-replace_outliers-{threshold}', labels_all)
    attampt_single(data, traces, [5 for i in range(steps-1)], steps, f'5-replace_outliers-{threshold}', labels_all)
    


In [20]:
get_result_replace(f['data'][:10000], traces_mean_outliers, 20, 2)

/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/

In [21]:
traces_mean_outliers = outlier_replace(f['traces'][:10000], 1.5)
print('\n')
get_result_replace(f['data'][:10000], traces_mean_outliers, 20, 1.5)

outlier number: 2137379
outlier ratio: 0.13169309919901417
traces_new.shape: (10000, 1623)




/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/home/yizhu/

In [ ]:
initials = [1623]
rates = rates = [0.9]
for initial in initials:
    for rate in rates:
        POIs = [initial]
        for i in range(88):
            POI = int(initial * rate ** (i+1) )

            if POI > 0:
                POIs.append(POI)
            else:
                continue

        if POIs:
            attampt(f['data'][:10000], traces_mean_outliers, POIs, len(POIs)+1, f'{initial}-{rate}-replace_outliers-2')

In [ ]:
data_no_outliers , traces_no_outliers = outlier_remove(f['data'][:10000], f['traces'][:10000],1.5)
print('\n')
#get_result_no(data_no_outliers, traces_no_outliers, 20)

initials = [1623,500,300,200,150,100,70,50,30,20,10,5]
rates = [0.9 - 0.1*i  for i in range(9)]
for initial in initials:
    for rate in rates:
        POIs = [initial]
        for i in range(38):
            POI = int(initial * rate ** (i+1) )

            if POI > 0:
                POIs.append(POI)
            else:
                continue

        if POIs:
            attampt(data_no_outliers, traces_no_outliers, POIs, len(POIs)+1, f'{initial}-{rate}-no_outliers-1.5')



traces_mean_outliers = outlier_replace(f['traces'][:10000], 1.5)
print('\n')
#get_result_replace(f['data'][:10000], traces_mean_outliers, 20)

initials = [1623,500,300,200,150,100,70,50,30,20,10,5]
rates = [0.9 - 0.1*i  for i in range(9)]
for initial in initials:
    for rate in rates:
        POIs = [initial]
        for i in range(38):
            POI = int(initial * rate ** (i+1) )

            if POI > 0:
                POIs.append(POI)
            else:
                continue

        if POIs:
            attampt(f['data'][:10000], traces_mean_outliers, POIs, len(POIs)+1, f'{initial}-{rate}-replace_outliers-1.5')

In [ ]:
POI_numbers = [int(f['traces'].shape[1] * 0.8**(i+1)) for i in range(30)]
print(POI_numbers)

In [ ]:
attampt(f['data'][:5000], f['traces'][:5000], POI_numbers, 30)
